In [1]:
# Importando as bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os
from ydata_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import pathlib
print(pathlib.Path)
import warnings
warnings.filterwarnings("ignore")

<class 'pathlib.Path'>


In [2]:
#Acessando o caminho do dataset
df_train = pd.read_csv('/home/moas/GembaGuard/data/bootcamp_train.csv', index_col=False)

In [3]:
# Converter e salvar como Parquet (mais eficiente)
df_train.to_parquet('/home/moas/GembaGuard/data/bootcamp_train.parquet', index=False)

In [4]:
df_train.head(25)

,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta,falha_maquina,FDF (Falha Desgaste Ferramenta),FDC (Falha Dissipacao Calor),FP (Falha Potencia),FTE (Falha Tensao Excessiva),FA (Falha Aleatoria)
0,0,L56434,L,298.3,309.1,90.0,1616.0,31.1,195.0,não,False,False,Não,False,Não
1,1,L48741,L,298.2,308.4,90.0,1388.0,53.8,137.0,Não,False,False,Não,False,Não
2,2,L48850,L,298.2,307.8,90.0,1528.0,31.1,NaN,Não,N,False,Não,False,Não
3,3,M20947,M,300.9,310.8,90.0,1599.0,33.0,7.0,não,False,False,Não,False,não
4,4,L53849,L,-36.0,310.5,90.0,1571.0,33.9,NaN,não,N,False,não,False,Não
5,5,L53113,L,300.5,310.5,90.0,-161.0,47.4,56.0,Não,False,False,Não,False,não
6,6,M23689,M,-36.0,308.8,90.0,1469.0,44.1,150.0,não,False,False,Não,False,não
7,7,L55125,L,300.8,311.7,90.0,1655.0,32.2,12.0,não,False,nao,Não,False,Não
8,8,L50688,L,301.7,310.9,90.0,1608.0,33.0,-202.0,não,False,False,não,False,Não
9,9,L49182,L,298.5,308.2,90.0,1654.0,27.1,13.0,Não,False,False,Não,False,não


In [5]:
# Conheceno o tamanho do meu dataset linhas e colunas
df_train.shape

(35260, 15)

In [6]:
#Analisando minhas features
df_train.dtypes

id                                   int64
id_produto                          object
tipo                                object
temperatura_ar                     float64
temperatura_processo               float64
umidade_relativa                   float64
velocidade_rotacional              float64
torque                             float64
desgaste_da_ferramenta             float64
falha_maquina                       object
FDF (Falha Desgaste Ferramenta)     object
FDC (Falha Dissipacao Calor)        object
FP (Falha Potencia)                 object
FTE (Falha Tensao Excessiva)          bool
FA (Falha Aleatoria)                object
dtype: object

In [7]:
df_train.head().T

,0,1,2,3,4
id,0,1,2,3,4
id_produto,L56434,L48741,L48850,M20947,L53849
tipo,L,L,L,M,L
temperatura_ar,298.3,298.2,298.2,300.9,-36.0
temperatura_processo,309.1,308.4,307.8,310.8,310.5
umidade_relativa,90.0,90.0,90.0,90.0,90.0
velocidade_rotacional,1616.0,1388.0,1528.0,1599.0,1571.0
torque,31.1,53.8,31.1,33.0,33.9
desgaste_da_ferramenta,195.0,137.0,NaN,7.0,NaN
falha_maquina,não,Não,Não,não,não


In [8]:
# Calcula a contagem nulos
df_train.isnull().sum()

id                                   0
id_produto                           0
tipo                                 0
temperatura_ar                     616
temperatura_processo               599
umidade_relativa                     0
velocidade_rotacional              751
torque                             623
desgaste_da_ferramenta             952
falha_maquina                        0
FDF (Falha Desgaste Ferramenta)      0
FDC (Falha Dissipacao Calor)         0
FP (Falha Potencia)                  0
FTE (Falha Tensao Excessiva)         0
FA (Falha Aleatoria)                 0
dtype: int64

In [9]:
# Observando a incidência de dados duplicados
duplicates = df_train.duplicated().sum()

In [10]:
#Analisando os valores únicos
for col in df_train.columns:
    print(f"{col}: {df_train[col].nunique()}")

id: 35260
id_produto: 9708
tipo: 3
temperatura_ar: 94
temperatura_processo: 83
umidade_relativa: 36
velocidade_rotacional: 916
torque: 579
desgaste_da_ferramenta: 247
falha_maquina: 8
FDF (Falha Desgaste Ferramenta): 6
FDC (Falha Dissipacao Calor): 6
FP (Falha Potencia): 8
FTE (Falha Tensao Excessiva): 2
FA (Falha Aleatoria): 7


In [11]:
#Análise da estatística descritiva dos nossos dados
df_train.describe()

,id,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta
count,35260.000000,34644.000000,34661.000000,35260.000000,34509.000000,34637.000000,34308.000000
mean,17629.500000,269.535241,280.457676,89.997672,1380.194181,40.210357,74.373266
std,10178.829582,96.342224,96.943630,0.142191,494.098759,8.831626,110.411937
min,0.000000,-36.000000,-38.000000,80.590429,-161.000000,3.800000,-202.000000
25%,8814.750000,297.900000,308.500000,90.000000,1408.000000,34.300000,28.000000
50%,17629.500000,299.600000,309.800000,90.000000,1483.000000,40.300000,94.000000
75%,26444.250000,301.100000,310.900000,90.000000,1574.000000,46.200000,155.000000
max,35259.000000,304.500000,313.800000,94.575256,2886.000000,76.600000,253.000000


In [12]:
# Conhecendo o nome das nossas colunas
df_train.columns

Index(['id', 'id_produto', 'tipo', 'temperatura_ar', 'temperatura_processo',
       'umidade_relativa', 'velocidade_rotacional', 'torque',
       'desgaste_da_ferramenta', 'falha_maquina',
       'FDF (Falha Desgaste Ferramenta)', 'FDC (Falha Dissipacao Calor)',
       'FP (Falha Potencia)', 'FTE (Falha Tensao Excessiva)',
       'FA (Falha Aleatoria)'],
      dtype='object')

In [13]:
profile = ProfileReport(
            df_train,
            title='EDA Report - Sistema de Manutenção Preditiva',
            explorative=True,
            minimal=False
        )

In [14]:
profile.to_file('/home/moas/GembaGuard/reports/eda_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:00<00:00, 409.78it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Para assegurar a integridade e a qualidade do dataset antes da modelagem, realizei uma análise diagnóstica que revelou a necessidade de intervenções específicas. A seguir, detalho os problemas identificados e a metodologia que aplicarei para corrigi-los.

- Diagnóstico de Multicolinearidade: Identifiquei uma forte correlação positiva entre as variáveis temperatura_ar e temperatura_processo, e também entre torque e velocidade_rotacional. Esta condição, conhecida como multicolinearidade, é problemática pois infla a variância dos coeficientes de estimação em modelos de regressão, comprometendo sua estabilidade e a interpretabilidade. Para quantificar este efeito, calcularei o Fator de Inflação de Variância (VIF) e, caso o valor exceda um limiar técnico (usualmente VIF > 5), removerei uma das variáveis do par colinear ou aplicarei a Análise de Componentes Principais (PCA) para gerar variáveis ortogonais.

- Tratamento de Alvos Multiclasse com Desbalanceamento: Verifiquei que as variáveis-alvo (falha_maquina, FDF, etc.) são de natureza categórica multiclasse. A análise de distribuição de frequência revelou um severo desbalanceamento entre as categorias. Treinar um modelo nestas condições levaria a um viés em favor das classes majoritárias, resultando em baixa capacidade de generalização para os eventos minoritários. Para mitigar isso, minha abordagem será aplicar uma técnica de sobreamostragem, como o SMOTE (Synthetic Minority Over-sampling Technique), para balancear o dataset de treinamento. Adicionalmente, explorarei o uso de funções de custo ponderadas (class_weight) nos algoritmos para penalizar mais fortemente os erros de classificação nas classes de menor frequência.

- Imputação de Valores Ausentes: Detectei a presença de valores ausentes (NaN) em preditores chave, incluindo torque e desgaste_da_ferramenta. A remoção dos registros afetados (listwise deletion) acarretaria uma perda substancial de dados. A imputação por medidas de tendência central, por sua vez, poderia suprimir a variância e distorcer as relações entre as variáveis. Portanto, optei por uma estratégia de imputação multivariada utilizando o algoritmo k-Nearest Neighbors (k-NN) Imputer, que estima os valores faltantes com base na estrutura local dos dados, preservando de forma mais eficaz as distribuições originais.

- Correção de Assimetria na Distribuição: A análise da variável umidade_relativa revelou um alto grau de assimetria (skewness). Esta característica viola premissas de normalidade em certos modelos estatísticos e pode impactar negativamente a performance de algoritmos sensíveis à escala. Para tratar esta questão, aplicarei a transformação de Yeo-Johnson. Escolhi este método por sua capacidade de lidar com dados que incluem valores não-positivos, garantindo maior generalidade em comparação com outras transformações de potência.

- Interpretação Semântica de Valores Zero: Identifiquei uma concentração de valores iguais a zero na variável desgaste_da_ferramenta. O valor zero neste contexto é ambíguo, podendo representar tanto uma ferramenta sem desgaste (nova) quanto uma falha de sensor. Tratar este valor como uma medida contínua pode introduzir um viés interpretativo no modelo. A estratégia a ser adotada é a engenharia de features: criarei uma nova variável binária, ferramenta_nova, que assumirá o valor 1 quando desgaste_da_ferramenta for zero e 0 caso contrário. Isso permite ao modelo aprender a relação específica deste estado inicial com as variáveis de falha.

- Remoção de Variáveis Não-Preditivas: A variável id é um identificador único para cada observação e, por definição, não possui valor preditivo intrínseco. Sua inclusão no conjunto de treinamento adicionaria dimensionalidade sem informação relevante, podendo ser interpretada como ruído. Portanto, esta variável será excluída do dataset antes da fase de modelagem.